In [23]:
import pandas as pd

In [24]:
df_posts = pd.read_json('../data/dataset.json')

In [25]:
df_posts['timestamp'] = pd.to_datetime(df_posts['timestamp'])

## Extract numeric features

### Temporal features

Number of posts over time

In [26]:
posts_per_user = df_posts.groupby('user')['timestamp'].count()

Time between consecutive posts

In [27]:
# Calculate the time difference between posts
df_posts['time_diff'] = df_posts.groupby('user')['timestamp'].diff()

# Convert time_diff to timedelta
df_posts['time_diff'] = pd.to_timedelta(df_posts['time_diff'], errors='coerce')

# Replace NaT with the median time difference
df_posts['time_diff'] = df_posts['time_diff'].fillna(df_posts['time_diff'].median())

In [28]:
# Convert time_diff to seconds
df_posts['time_diff_seconds'] = df_posts['time_diff'].dt.total_seconds()

# Drop the time_diff column
df_posts = df_posts.drop(columns=['time_diff'])

### Text-based features

Tweet length

In [29]:
df_posts['text_length'] = df_posts['text'].apply(len)

Word count

In [30]:
df_posts['word_count'] = df_posts['text'].apply(lambda x: len(x.split()))

Sentiment Analysis score

Keyword count

Hashtag count

Topics from BERTopic

### Network Features

Friend count

Clustering Coefficient (how connected a user's neighbors are in the network)

Influence Score

### Aggregate Features per User

Total Poster per User

Average Post Sentiment Per User

Variability in Posting Times

### Combined Features

Posting Behavior + Text Content: How does sentiment correlate with posting frequency or time?

Network + Text Analysis: Do influential users post more positive content?

- Normalize derived numerical features for clustering 
- Use features for clustering(k-Means or hierarchical)